In [16]:
import re
import pandas as pd

from mozilla_sec_eia.utils import GCSArchive

In [ ]:
archive = GCSArchive()
md = archive.get_metadata()

In [ ]:
blob = archive.get_blob()

# Demo with one filing

In [12]:
import os
from pathlib import Path

In [128]:
columns=["company_name",
         "cik",
         "stand_ind_class",
         "irs_num",
         "state_of_inc",
         "fiscal_year_end",
         "sec_file_num",
         "film_num",
         "biz_street_1",
         "biz_street_2",
         "biz_city",
         "biz_state",
         "biz_zip",
         "phone",
         "mail_street_1",
         "mail_street_2",
         "mail_city",
         "mail_state",
         "mail_zip"
        ]

In [376]:
company_regex = r"""\s*COMPANY CONFORMED NAME:\s*(?P<company_name>[\S ]+)
\s*CENTRAL INDEX KEY:\s*(?P<cik>[\d]+)
\s*STANDARD INDUSTRIAL CLASSIFICATION:\s*(?P<stand_ind_class>[\S ]+)
\s*IRS NUMBER:\s*(?P<irs_num>[\d]+)
?(?:\s*(?:\s*STATE OF INCORPORATION:\s*(?P<state_of_inc>[\w]{2}))?)?
\s*FISCAL YEAR END:\s*(?P<fiscal_year_end>[\d]{3,4})"""

filing_regex = r"""\s*FILING VALUES:\s*
\s*FORM TYPE:\s*10-K
\s*SEC ACT:\s*(?P<sec_act>[\S ]+)?
\s*SEC FILE NUMBER:\s*(?P<sec_file_num>[\d-]+)
\s*FILM NUMBER:\s*(?P<film_num>[\d]+)"""

business_regex = r"""\s*BUSINESS ADDRESS:\s*
\s*STREET 1:\s*(?P<biz_street_1>[\S ]+)
?(?:\s*STREET 2:\s*(?P<biz_street_2>[\S ]*))?
\s*CITY:\s*(?P<biz_city>[\S ]+)
\s*STATE:\s*(?P<biz_state>[\w]{2})
\s*ZIP:\s*(?P<biz_zip>[\S ]+)
?(?:\s*BUSINESS PHONE:\s*(?P<phone>[\d\-]+))?"""

mailing_regex = r"""\s*MAIL ADDRESS:\s*
\s*STREET 1:\s*(?P<mail_street_1>[\S ]+)
?(?:\s*STREET 2:\s*(?P<mail_street_2>[\S ]+))?
\s*CITY:\s*(?P<mail_city>[\S ]+)
\s*STATE:\s*(?P<mail_state>[\w]{2})
\s*ZIP:\s*(?P<mail_zip>[\S ]+)"""

former_company_regex = r"""\s*FORMER COMPANY:\s*
\s*FORMER CONFORMED NAME:\s*(?P<former_name>[\S ]+)
\s*DATE OF NAME CHANGE:\s*(?P<date_of_change>[\d ]+)"""

company_header = r"COMPANY DATA:"
filing_header = r"FILING VALUES:"
business_header = r"BUSINESS ADDRESS:"
mailing_header = r"MAIL ADDRESS:"
former_company_header = r"FORMER COMPANY:"

In [377]:
regex_sections = {"company": company_regex, 
                  "filing": filing_regex,
                  "business address": business_regex,
                  "mailing address": mailing_regex,
                  "former_company": former_company_regex
                 }

In [374]:
header_sections = {"company": company_header,
                   "filing": filing_header,
                   "business address": business_header,
                   "mailing address": mailing_header,
                   "former_company": former_company_header
                  }

In [347]:
d = re.findall(former_company_regex, s)
if not d:
    print("hi")

hi


In [384]:
cache_dir = Path("../sec10k_filings")
extracted_data = []
missed_docs = set()
n_all_docs = 0
for filename in os.listdir(cache_dir):
    if filename.split(".")[-1] == "html":
        filepath = cache_dir / filename
        with open(filepath, "r") as file:
            txt = file.read()
            data_dict = {"filename": filename}
            for section_name, regex in regex_sections.items():
                if section_name == "former_company":
                    match = re.findall(regex, txt)
                    if match:
                        match_dict = {"former_companies": [group[0] for group in match],
                                      "dates_of_change": [group[1] for group in match]}
                        data_dict = dict(data_dict, **match_dict)
                else:
                    match = re.search(regex, txt)
                    if match:
                        match_dict = match.groupdict()
                        data_dict = dict(data_dict, **match_dict)  
                if not match:
                    header_match = re.search(header_sections[section_name], txt)
                    if header_match:
                        print(f"Could not extract {section_name} data for {filename}")
                        missed_docs.add(filename)
            n_all_docs += 1
            extracted_data.append(data_dict)
df = pd.DataFrame(extracted_data)
print(f"percent docs missed: {len(missed_docs)/n_all_docs}")

percent docs missed: 0.0


In [379]:
df

,filename,company_name,cik,stand_ind_class,irs_num,state_of_inc,fiscal_year_end,sec_act,sec_file_num,film_num,...,biz_state,biz_zip,phone,mail_street_1,mail_street_2,mail_city,mail_state,mail_zip,former_companies,dates_of_change
0,107815-2017q1-107815-0000107815-17-000106.html,WISCONSIN ELECTRIC POWER CO,0000107815,ELECTRIC SERVICES [4911],390476280,WI,1231,1934 Act,001-01245,17646142,...,WI,53290-0001,414-221-2345,231 W MICHIGAN ST,PO BOX 2046,MILWAUKEE,WI,53201,NaN,NaN
1,12208-2013q1-12208-0000012208-13-000007.html,BIO RAD LABORATORIES INC,0000012208,LABORATORY ANALYTICAL INSTRUMENTS [3826],941381833,DE,1231,1934 Act,001-07928,13697997,...,CA,94547,5107247000,1000 ALFRED NOBEL DR,None,HERCULES,CA,94547,NaN,NaN
2,1277250-2022q1-1277250-0001654954-22-003967.html,"Charge Enterprises, Inc.",0001277250,TELEGRAPH & OTHER MESSAGE COMMUNICATIONS [4822],900471969,DE,1231,1934 Act,333-253073,22777861,...,NY,10017,917-268-0660,"125 PARK AVENUE, 25TH FLOOR",None,NEW YORK,NY,10017,"[TransWorld Holdings, Inc., GoIP GLOBAL, INC.,...","[20201005, 20050816, 20040122]"
3,1591698-2018q3-1591698-0001558370-18-006955.html,Paylocity Holding Corp,0001591698,SERVICES-PREPACKAGED SOFTWARE [7372],464066644,DE,0630,1934 Act,001-36348,181008099,...,IL,60173,800-520-2687,1400 AMERICAN LANE,None,SCHAUMBURG,IL,60173,NaN,NaN
4,1096481-2010q1-1096481-0001193125-10-058537.html,ANGIOTECH PHARMACEUTICALS INC,0001096481,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS [3841],000000000,None,1231,1934 Act,000-30334,10686274,...,A1,V6A 1B6,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,941138-2000q1-941138-0000100122-00-000004.html,TUCSON ELECTRIC POWER CO,0000100122,ELECTRIC SERVICES [4911],860062700,AZ,1231,None,001-05924,563708,...,AZ,85701,5025714000,220 WEST SIXTH STREET,P O BOX 711,TUCSON,AZ,87701,[TUCSON GAS & ELECTRIC CO /AZ/],[19790528]
6,1177274-2010q1-1177274-0001079973-10-000019.html,Rodobo International Inc,0001177274,DAIRY PRODUCTS [2020],752980786,NV,0930,1934 Act,000-50340,10525486,...,F4,150001,011-86-045182260522,380 CHANGJIANG ROAD,None,"NANGANG DISTRICT, HARBIN",F4,150001,"[Navstar Media Holdings, Inc., PREMIER DOCUMEN...","[20051206, 20020711]"
